# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

Шаги исследования:
1. [Открытие данных](#start)
2. [Разбить на выборки](#split)
3. [Исследовать модели](#model)
4. [Проверить модель на тестовой выборке](#check)
5. [Проверить модель на адекватность](#bonus)
6. [Общий вывод](#output)

<a id="start"></a>
## Откройте и изучите файл

In [26]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV #NEW
from sklearn.model_selection import RandomizedSearchCV #NEW
from IPython.display import display

users_behavior = pd.read_csv('/datasets/users_behavior.csv', sep=",")
print('--------------')
print('Users behavior')
print('--------------')
display(users_behavior.head(50))
display(users_behavior.describe())
users_behavior.info()

--------------
Users behavior
--------------


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


<a id="split"></a>
## Разбейте данные на выборки

In [27]:
features = users_behavior.drop(['is_ultra'], axis=1)
target = users_behavior['is_ultra']
#Выделим обучающую выборку
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345)
#Выделим вылидационную и тестовую выборки
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345)
print('Размер обучающей выборки',features_train.count()) #NEW
print('Размер вылидационной выборки',features_valid.count()) #NEW
print('Размер тестовой выборки',features_test.count()) #NEW

Размер обучающей выборки calls       1928
minutes     1928
messages    1928
mb_used     1928
dtype: int64
Размер вылидационной выборки calls       643
minutes     643
messages    643
mb_used     643
dtype: int64
Размер тестовой выборки calls       643
minutes     643
messages    643
mb_used     643
dtype: int64


<a id="model"></a>
## Исследуйте модели

In [31]:
#Проверим модель логистической регрессии
model_reg = LogisticRegression(random_state = 12345)#модель логистической регрессии
print(model_reg)
model_reg.fit(features_train, target_train) #модель на тренировочной выборке
result_reg = model_reg.score(features_valid, target_valid) #метрика качества модели на валидационной выборке
print("Accuracy модели логистической регрессии на валидационной выборке:", result_reg)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
Accuracy модели логистической регрессии на валидационной выборке: 0.7589424572317263


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [29]:
#Выберем лучшую модель случайного леса
best_model = None
best_result = 0
for est in range(1, 11):
    model_tree = RandomForestClassifier(random_state=12345, n_estimators= est) #модель с заданным количеством деревьев
    model_tree.fit(features_train, target_train) # модель на тренировочной выборке
    result_tree = model_reg.score(features_valid, target_valid) # качество модели на валидационной выборке
    if result_tree > best_result:
        best_model = model_tree#наилучшая модель
        best_result = result_tree#наилучшее значение метрики accuracy на валидационных данных

print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print(best_model)

Accuracy наилучшей модели на валидационной выборке: 0.7589424572317263
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)


<div class="alert alert-block alert-info">

<b>Совет:</b> 
Можно также применить GridSearchCv и RandomizedSearchCV
    
Есть хороший пример: https://www.mygreatlearning.com/blog/gridsearchcv/
</div>

<a id="check"></a>
## Проверьте модель на тестовой выборке

In [30]:
model_tree_test = RandomForestClassifier(random_state=12345, n_estimators= 10)# инициализирую модель RandomForestRegressor с параметрами random_state=12345, n_estimators=est и max_depth=depth
model_tree_test.fit(features_train,target_train) 
#result = model.score(features_valid, target_valid)
result_model_tree_test = model_tree_test.score(features_test, target_test)
result_reg = model_reg.score(features_test, target_test)
print("Accuracy модели логистической регрессии на тестовой выборке:", result_reg)
print("Accuracy модели случайного леса на тестовой выборке:", result_model_tree_test)
prediction = pd.merge(features_test, target_test, left_index=True, right_index=True)
display(prediction.head(5))

Accuracy модели логистической регрессии на тестовой выборке: 0.7402799377916018
Accuracy модели случайного леса на тестовой выборке: 0.7807153965785381


,calls,minutes,messages,mb_used,is_ultra
160,61.0,495.11,8.0,10891.23,0
2498,80.0,555.04,28.0,28083.58,0
1748,87.0,697.23,0.0,8335.70,0
1816,41.0,275.80,9.0,10032.39,0
1077,60.0,428.49,20.0,29389.52,1


С учетом проверки на тестовой выборке следует выбрать модель случайного леса с n_estimators = 10

<a id="bonus"></a>
## (бонус) Проверьте модели на адекватность

In [25]:
#Люди, которые часто разговоривают, шлют сообщения и используют интернет, выберут тариф "Ультра"
#calls	minutes	messages	mb_used
new_features = pd.DataFrame(
    [[20000.0, 2000.0, 2000000.0, 20000.0]],
    columns=features.columns)
predictions = model_tree_test.predict(new_features)
print(predictions)

from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train, target_train) # модель на тренировочной выборке
dummy_clf.predict(features_valid)
result_dummy = dummy_clf.score(features_valid, target_valid)
print('Accuracy DummyClassifier на тестовой выборке:', result_dummy)

[1]
Accuracy DummyClassifier на тестовой выборке: 0.7060653188180405


<div class="alert alert-block alert-info">

<b>Вывод:</b> 
Полученная модель предсказывает лучше, чем DummyClassifier.
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
